In [7]:
import torch
import time
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print("当前设备名称：", torch.cuda.get_device_name(device) if device.type == 'cuda' else 'CPU')

> 生成词索引构成的序列

In [8]:
batch_size = 2
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8
max_src_seq_len = 5
max_tgt_seq_len = 5
max_position_len = 5
# src_len = torch.randint(2,5,(batch_size,))
# tgt_len = torch.randint(2,5,(batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)
# 词索引构成的序列
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max_src_seq_len-L)), 0) \
                     for L in src_len])
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max_tgt_seq_len-L)), 0) \
                     for L in tgt_len])
print(src_seq)
print(tgt_seq)

tensor([[3, 6, 0, 0, 0],
        [5, 3, 2, 3, 0]])
tensor([[6, 7, 2, 6, 0],
        [5, 5, 6, 0, 0]])


> 采用Embedding的forward方法

In [9]:
# 实例化embedding # requires_grad 
# weight是随机的，训练时更新
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)
# print(src_embedding_table.weight, tgt_embedding_table.weight)
# Embedding存在 --call-- forward方法隐式
src_embedding = src_embedding_table(src_seq)
tgt_embedding_table = tgt_embedding_table(tgt_seq)
print(src_embedding)
# 由于为0 底下3排一样

tensor([[[-0.4833, -0.7190, -0.8644,  0.0754, -2.3833, -1.0124, -0.6498,
          -0.6566],
         [-1.3059,  2.1022, -1.1969,  0.6897,  2.1104, -1.2545, -2.0167,
           1.5300],
         [-0.3475, -0.6750,  0.8758, -0.2361,  0.6011, -1.1490,  1.3559,
           0.7823],
         [-0.3475, -0.6750,  0.8758, -0.2361,  0.6011, -1.1490,  1.3559,
           0.7823],
         [-0.3475, -0.6750,  0.8758, -0.2361,  0.6011, -1.1490,  1.3559,
           0.7823]],

        [[ 2.0757, -1.0883, -1.4361, -1.8673,  1.2355,  0.1206, -0.5114,
          -2.2540],
         [-0.4833, -0.7190, -0.8644,  0.0754, -2.3833, -1.0124, -0.6498,
          -0.6566],
         [-0.8011, -1.1284, -0.3874, -0.5417, -0.6098,  0.6973, -0.6546,
           1.5442],
         [-0.4833, -0.7190, -0.8644,  0.0754, -2.3833, -1.0124, -0.6498,
          -0.6566],
         [-0.3475, -0.6750,  0.8758, -0.2361,  0.6011, -1.1490,  1.3559,
           0.7823]]], grad_fn=<EmbeddingBackward>)


> 生成pos—embedding

In [10]:
# pos embedding    method1 
start_time = time.time()
pos_embedding = torch.zeros(max_position_len, model_dim)
for i in range(0, model_dim):
    for j in range(0, max_position_len):
        if i%2 == 0:
            # print(i,j)
            pos_embedding[j, i] = np.sin(j/np.power(10000, (i/model_dim)))
        else:
            pos_embedding[j, i] = np.cos(j/np.power(10000, (i/model_dim)))
# print(pos_embedding)
end_time = time.time()
# 计算执行时间并转换为微秒
execution_time_microseconds = (end_time - start_time) * 1e6
# 打印执行时间
print("np Time (microseconds):", execution_time_microseconds)
#///////////////////////////////////////////////////////////////////////////////////////////
# pos embedding    method2 
start_time = time.time()
pos_mat = torch.arange(max_position_len).reshape(-1, 1) 
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape((1, -1))/model_dim)
i1_mat = torch.pow(10000, torch.arange(1, 8, 2).reshape((1, -1))/model_dim)
# print(pos_mat, i_mat, i1_mat)
pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat/i_mat)
pe_embedding_table[:, 1::2] = torch.cos(pos_mat/i1_mat)
print(pe_embedding_table)
end_time = time.time()
# 计算执行时间并转换为微秒
execution_time_microseconds = (end_time - start_time) * 1e6
# 打印执行时间
print("torch Time (microseconds):", execution_time_microseconds)
#///////////////////////////////////////////////////////////////////////////////////////////
# print(torch.sub(pos_embedding, pe_embedding_table))
pe_embedding = nn.Embedding(max_position_len, model_dim)
# 位置编码训练中不发生变化requires_grad=False
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
# 注意 这里给embedding的是位置 不是词的编号
# src_p=torch.zeros_like(src_seq)
# for i in range(src_p.size(0)):
#     for j in range(src_p.size(1)):
#         src_p[i][j] = j 
# print(src_p)
# src_pe_embedding =pe_embedding(src_p)
src_pe_embedding =pe_embedding(torch.cat([pos_mat,pos_mat], 1).transpose(-1,-2))
print(torch.cat([pos_mat,pos_mat], 1).transpose(-1,-2))
# print(src_pe_embedding)
tgt_pe_embedding = src_pe_embedding

np Time (microseconds): 999.9275207519531
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
          9.9999e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01,  8.0658e-01,  1.9867e-01,  9.9800e-01,  1.9999e-02,
          9.9998e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01,  5.8275e-01,  2.9552e-01,  9.9550e-01,  2.9995e-02,
          9.9995e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01,  3.0114e-01,  3.8942e-01,  9.9201e-01,  3.9989e-02,
          9.9992e-01,  4.0000e-03,  1.0000e+00]])
torch Time (microseconds): 2000.570297241211
tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])


> 生成enc_self_attn_mask 无因果关系   对词向量相似度进行masked

In [11]:
# 这里不是三角矩阵# 这里不是三角矩阵 没有因果关系 多用矩阵相乘 转置 升维
vaild_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len]), 2)
# print(vaild_encoder_pos.shape, "\n", vaild_encoder_pos)
# vaild_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len])
print(vaild_encoder_pos.shape, "\n", vaild_encoder_pos)
# 这里不是三角矩阵
# print(torch.tril(torch.ones(4, 4)))
# vaild_mask = vaild_encoder_pos.transpose(1,2)*torch.ones(4, 4)*vaild_encoder_pos
# print(vaild_mask.shape, "\n", vaild_mask)
# print(torch.bmm(torch.unsqueeze(torch.ones(4, 4),0), torch.unsqueeze(torch.ones(4, 4),0)))
# print(torch.ones(4, 4)*torch.ones(4, 4))
valid_encoder_pos_matrix = torch.bmm(vaild_encoder_pos, vaild_encoder_pos.transpose(1, 2))
invalid_encoder_pos_matrix = 1-valid_encoder_pos_matrix
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
print(mask_encoder_self_attention)
#假设是vk注意力权重
score = torch.randn(batch_size, max(src_len), max(src_len))
masked_score = score.masked_fill(mask_encoder_self_attention, -np.inf)
prob = F.softmax(masked_score, -1)
print(prob)

torch.Size([2, 4, 1]) 
 tensor([[[1.],
         [1.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.]]])
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])
tensor([[[0.1922, 0.8078, 0.0000, 0.0000],
         [0.4465, 0.5535, 0.0000, 0.0000],
         [   nan,    nan,    nan,    nan],
         [   nan,    nan,    nan,    nan]],

        [[0.0453, 0.4057, 0.3377, 0.2113],
         [0.7340, 0.0341, 0.0798, 0.1520],
         [0.5906, 0.2495, 0.1014, 0.0586],
         [0.0880, 0.0214, 0.2059, 0.6847]]])


>构造intra-attention的mask

In [12]:
# shape: [batch_size, tgt_seq_len, src_seq_len]
vaild_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len]), 2)